In [ ]:
import json
import pandas as pd
from datetime import datetime as dt
from typing import Union
import numpy as np
import ScraperFC as sfc
ss = sfc.Sofascore()
np.seterr(all='ignore')

In [ ]:
all_leagues = ['EPL', 'La Liga', 'Bundesliga', 'Serie A', 'Ligue 1']
all_columns = ['goals', 'yellowCards', 'redCards', 'groundDuelsWon',
       'groundDuelsWonPercentage', 'aerialDuelsWon',
       'aerialDuelsWonPercentage', 'successfulDribbles',
       'successfulDribblesPercentage', 'tackles', 'assists',
       'accuratePassesPercentage', 'totalDuelsWon', 'totalDuelsWonPercentage',
       'minutesPlayed', 'wasFouled', 'fouls', 'dispossessed', 'appearances',
       'saves', 'savedShotsFromInsideTheBox', 'savedShotsFromOutsideTheBox',
       'goalsConcededInsideTheBox', 'goalsConcededOutsideTheBox', 'accurateFinalThirdPasses',
       'bigChancesCreated', 'accuratePasses', 'keyPasses', 'accurateCrosses',
       'accurateCrossesPercentage', 'accurateLongBalls',
       'accurateLongBallsPercentage', 'interceptions', 'clearances',
       'dribbledPast', 'bigChancesMissed', 'totalShots', 'shotsOnTarget',
       'blockedShots', 'goalConversionPercentage', 'hitWoodwork', 'offsides',
       'expectedGoals', 'errorLeadToGoal', 'errorLeadToShot', 'passToAssist',
       'player', 'team', 'player id', 'team id']

f_test_elimination_value = 10
season = "24/25"

player_data = []
for league in all_leagues[:5]:
    player_data.append(ss.scrape_player_league_stats(season, league))
    print("Fetched data for {} and season {}".format(league, season))
original_player_data = pd.concat(player_data)[all_columns].fillna(0).reset_index(drop=True)
player_data = pd.concat(player_data[2:5])[all_columns].fillna(0)
player_data.head()

In [ ]:
from analytics import Analytics, ClubAnalytics, PlayerAnalytics

analysis_goals = Analytics(player_data)
final_features = analysis_goals.run_regression_and_feature_selection('goals', 10)
final_prediction_data = original_player_data[final_features]
club_analysis_goals = ClubAnalytics(original_player_data[final_features])
display(final_prediction_data.head())
analysis_goals.save_most_similar_players_list('similar_player_goals.json', 5, final_prediction_data)

In [ ]:
analysis_assists = Analytics(player_data)
final_features = analysis_assists.run_regression_and_feature_selection('assists', 10)
final_prediction_data = original_player_data[final_features]
club_analysis_assists = ClubAnalytics(original_player_data[final_features])
display(final_prediction_data.head())
analysis_assists.save_most_similar_players_list('similar_player_assists.json', 5, final_prediction_data)

In [ ]:
analysis_fouls = Analytics(player_data)
final_features = analysis_fouls.run_regression_and_feature_selection('fouls', 10)
final_prediction_data = original_player_data[final_features]
club_analysis_fouls = ClubAnalytics(original_player_data[final_features])
display(final_prediction_data.head())
analysis_fouls.save_most_similar_players_list('similar_player_fouls.json', 5, final_prediction_data)

In [ ]:
print(analysis_assists.predict(original_player_data[original_player_data['player'] == 'Cole Palmer']).to_list()[0])

print(original_player_data[original_player_data['player'] == 'Cole Palmer']['assists'].to_list()[0])

## CLUB BASED PREFORMANCE

In [ ]:
club_analysis_goals.get_players_of_club('Manchester United')

In [ ]:
club_analysis_goals.get_club_of_players(['Marcus Rashford', 'Kevin De Bruyne'])

## PER PLAYER ANALYSIS OF PERFORMANCE (PER SEASON)

In [1]:
from sleeping_bets.analytics import PlayerAnalytics
import json
import json
import pandas as pd
from datetime import datetime as dt
from typing import Union
import numpy as np
import ScraperFC as sfc
ss = sfc.Sofascore()
np.seterr(all='ignore')
season = "24/25"

with open('similar_player_assists.json', 'r', encoding='utf-8') as file:
    similar_assists = json.load(file)

with open('similar_player_goals.json', 'r', encoding='utf-8') as file:
    similar_goals = json.load(file)

with open('similar_player_fouls.json', 'r', encoding='utf-8') as file:
    similar_fouls = json.load(file)

In [8]:
player_name = 'Marcus Rashford'
player_league= 'EPL'
simliar_players = [x['player'] for x in similar_assists[player_name]]

In [ ]:
player_comp = PlayerAnalytics(simliar_players, ss, season)

In [ ]:
similar_player_data = player_comp.get_historic_similarity_stats()

In [ ]:
player_stats = player_comp.get_historic_player_stats(player_name, player_league)

In [ ]:
comparable_values = player_stats.columns
print(comparable_values)

In [ ]:
def plot_comarison(field, similar_players_data: dict, player_data, lead_player):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 6))
    for player in similar_players_data.keys():
        try:
            plt.plot(similar_players_data[player]['matchTime'],
                similar_players_data[player][field],
                label=player)
            for x, y in zip(similar_players_data[player]['matchTime'], similar_players_data[player][field]):
                plt.text(x, y, f'{y}', fontsize=8, ha='right')
        except KeyError:
            print("The player data not available for {} state: {}".format(player, field))
    plt.xlabel('Date')
    plt.ylabel(field)
    plt.title('Comparison of {} for player and similar players'.format(field))
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

plot_comarison('touches', similar_player_data, player_stats, player_name)